## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials configured
* Amazon Bedrock AgentCore SDK
* MCP (Model Context Protocol) library
* Docker running

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Understanding MCP (Model Context Protocol)

MCP is a protocol that allows AI models to securely access external data and tools. Key concepts:

* **Tools**: Functions that the AI can call to perform actions
* **Streamable HTTP**: Transport protocol used by AgentCore Runtime
* **Session Isolation**: Each client gets isolated sessions via `Mcp-Session-Id` header
* **Stateless Operation**: Servers must support stateless operation for scalability

AgentCore Runtime expects MCP servers to be hosted on `0.0.0.0:8000/mcp` as the default path.

### Project Structure

Let's set up our project with the proper structure:

```
mcp_server_project/
├── mcp_server.py              # Main MCP server code
├── my_mcp_client.py          # Local testing client
├── my_mcp_client_remote.py   # Remote testing client
├── requirements.txt          # Dependencies
└── __init__.py              # Python package marker
```

## Creating MCP Server

Now let's create our MCP server with three simple tools. The server uses FastMCP with `stateless_http=True` which is required for AgentCore Runtime compatibility.

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse
from typing import Dict, List
import uuid

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

# In-memory storage
rapps_db: Dict[str, dict] = {
    "qos-optimizer": {
        "rappId": "qos-optimizer",
        "name": "QoS Optimizer rApp",
        "state": "PRIMED",
        "reason": "Successfully primed and ready for instantiation",
        "packageLocation": "/packages/qos-optimizer-v1.0.csar",
        "packageName": "qos-optimizer-v1.0.csar",
        "rappInstances": {}
    }
}

@mcp.tool()
def get_rapps() -> List[dict]:
    """Get all rApps"""
    return list(rapps_db.values())

@mcp.tool()
def create_rapp(package_name: str) -> dict:
    """Create a new rApp"""
    rapp_id = f"rapp-{uuid.uuid4().hex[:8]}"
    new_rapp = {
        "rappId": rapp_id,
        "name": f"rApp {rapp_id}",
        "state": "COMMISSIONED",
        "reason": "rApp package uploaded and validated",
        "packageLocation": f"/packages/{package_name}",
        "packageName": package_name,
        "rappInstances": {}
    }
    rapps_db[rapp_id] = new_rapp
    return {"rappId": rapp_id, "message": "rApp created successfully"}

@mcp.tool()
def get_rapp(rapp_id: str) -> dict:
    """Get rApp by ID"""
    if rapp_id not in rapps_db:
        return {"error": "rApp not found"}
    return rapps_db[rapp_id]

@mcp.tool()
def delete_rapp(rapp_id: str) -> dict:
    """Delete rApp"""
    if rapp_id not in rapps_db:
        return {"error": "rApp not found"}
    del rapps_db[rapp_id]
    return {"message": "rApp deleted successfully"}

@mcp.tool()
def prime_rapp(rapp_id: str, prime_order: str) -> dict:
    """Prime or deprime rApp"""
    if rapp_id not in rapps_db:
        return {"error": "rApp not found"}
    
    rapp = rapps_db[rapp_id]
    if prime_order == "PRIME":
        rapp["state"] = "PRIMED"
        rapp["reason"] = "rApp primed successfully"
    else:
        rapp["state"] = "COMMISSIONED"
        rapp["reason"] = "rApp deprimed successfully"
    
    return {"message": f"rApp {prime_order.lower()} operation accepted"}

@mcp.tool()
def get_rapp_instances(rapp_id: str) -> List[dict]:
    """Get rApp instances"""
    if rapp_id not in rapps_db:
        return []
    
    rapp = rapps_db[rapp_id]
    return list(rapp["rappInstances"].values())

@mcp.tool()
def create_rapp_instance(rapp_id: str, instance_id: str = None) -> dict:
    """Create rApp instance"""
    if rapp_id not in rapps_db:
        return {"error": "rApp not found"}
    
    rapp = rapps_db[rapp_id]
    if rapp["state"] != "PRIMED":
        return {"error": "rApp must be primed before creating instances"}
    
    if not instance_id:
        instance_id = f"instance-{uuid.uuid4().hex[:8]}"
    
    new_instance = {
        "rappInstanceId": instance_id,
        "state": "UNDEPLOYED",
        "reason": "Instance created successfully"
    }
    
    rapp["rappInstances"][instance_id] = new_instance
    return {"rappInstanceId": instance_id, "message": "Instance created successfully"}

@mcp.tool()
def get_rapp_instance(rapp_id: str, instance_id: str) -> dict:
    """Get rApp instance"""
    if rapp_id not in rapps_db:
        return {"error": "rApp not found"}
    
    rapp = rapps_db[rapp_id]
    if instance_id not in rapp["rappInstances"]:
        return {"error": "Instance not found"}
    
    return rapp["rappInstances"][instance_id]

@mcp.tool()
def delete_rapp_instance(rapp_id: str, instance_id: str) -> dict:
    """Delete rApp instance"""
    if rapp_id not in rapps_db:
        return {"error": "rApp not found"}
    
    rapp = rapps_db[rapp_id]
    if instance_id not in rapp["rappInstances"]:
        return {"error": "Instance not found"}
    
    del rapp["rappInstances"][instance_id]
    return {"message": "Instance deleted successfully"}

@mcp.tool()
def deploy_rapp_instance(rapp_id: str, instance_id: str, deploy_order: str) -> dict:
    """Deploy or undeploy instance"""
    if rapp_id not in rapps_db:
        return {"error": "rApp not found"}
    
    rapp = rapps_db[rapp_id]
    if instance_id not in rapp["rappInstances"]:
        return {"error": "Instance not found"}
    
    instance = rapp["rappInstances"][instance_id]
    if deploy_order == "DEPLOY":
        instance["state"] = "DEPLOYED"
        instance["reason"] = "Instance deployed successfully"
    else:
        instance["state"] = "UNDEPLOYED"
        instance["reason"] = "Instance undeployed successfully"
    
    return {"message": f"Instance {deploy_order.lower()} operation accepted"}

if __name__ == "__main__":
    mcp.run(transport="streamable-http")


### What This Code Does

* **FastMCP**: Creates an MCP server that can host your tools
* **@mcp.tool()**: Decorator that turns your Python functions into MCP tools
* **stateless_http=True**: Required for AgentCore Runtime compatibility
* **Tools**: Three simple tools demonstrating different types of operations

## Creating Local Testing Client

Before deploying to AgentCore Runtime, let's create a client to test our MCP server locally:

In [ ]:
%%writefile my_mcp_client.py
import asyncio

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

### Testing Locally

To test your MCP server locally:

1. **Terminal 1**: Start the MCP server
   ```bash
   python mcp_server.py
   ```
   
2. **Terminal 2**: Run the test client
   ```bash
   python my_mcp_client.py
   ```

You should see your three tools listed in the output.

## Setting up Amazon Cognito for Authentication

AgentCore Runtime requires authentication. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server.

In [ ]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from cognito_utils import create_agentcore_role, setup_cognito_user_pool

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

## Step 5: Create IAM Execution Role

Before starting, let's create an IAM role for our AgentCore Runtime. This role provides the necessary permissions for the runtime to operate.

In [ ]:
tool_name = "mcp_server_r1"
print(f"Creating IAM role for {tool_name}...")
agentcore_iam_role = create_agentcore_role(agent_name=tool_name)
print(f"IAM role created ✓")
print(f"Role ARN: {agentcore_iam_role['Role']['Arn']}")

## Configuring AgentCore Runtime Deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()


auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=tool_name
)
print("Configuration completed ✓")

In [ ]:
agentcore_iam_role['Role']['Arn']

In [ ]:
auth_config

## Launching MCP Server to AgentCore Runtime

Now that we've got a docker file, let's launch the MCP server to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime.

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

## Verify Deployment

To verify your MCP server was successfully deployed:

1. Go to the **AWS Console** search bar
2. Search for **AgentCore**
3. In the left navigation bar, click **AgentCore Runtime**
4. Look for a runtime named **`mcp_server_r1`** in the list
5. The status should show **CREATING** initially, then **READY** when deployment is complete

:::alert{type="info"}
**Note:** Deployment typically takes 3-5 minutes to complete.
:::

Let's also verify the deployment programmatically and wait for it to be ready.

## Checking AgentCore Runtime Status Programmatically

By now, the runtime should be ready. Let's verify the status programmatically:

In [ ]:
print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

## Storing Configuration for Remote Access

Before we can invoke our deployed MCP server, let's store the Agent ARN and Cognito configuration in AWS Systems Manager Parameter Store and AWS Secrets Manager for easy retrieval:

In [ ]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/r1/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/r1/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/r1/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

client_id_response = ssm_client.put_parameter(
    Name='/mcp_server/r1/runtime/client_id',
    Value=cognito_config["client_id"],
    Type='String',
    Description='Client ID for auth',
    Overwrite=True
)

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")
cognito_config

## Creating Remote Testing Client

Now let's create a client to test our deployed MCP server. This client will retrieve the necessary credentials from AWS and connect to the deployed server:

In [ ]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/r1/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/r1/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)

        cognito_client = boto3.client('cognito-idp', region_name=region)   
        auth_response = cognito_client.initiate_auth(
                    ClientId=ssm_client.get_parameter(Name='/mcp_server/r1/runtime/client_id')["Parameter"]["Value"],
                    AuthFlow='USER_PASSWORD_AUTH',
                    AuthParameters={
                        'USERNAME': 'testuser',
                        'PASSWORD': 'MyPassword123!'
                    }
                )
        bearer_token=auth_response['AuthenticationResult']['AccessToken']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    print("HEADERS ARE:\n")
    print(headers)
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## Testing Your Deployed MCP Server

Let's test our deployed MCP server using the remote client:

In [ ]:
print("Testing deployed MCP server...")
print("=" * 50)
!python my_mcp_client_remote.py

## Invoking MCP Tools Remotely

Now let's create an enhanced client that not only lists tools but also invokes them to demonstrate the full MCP functionality:

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/r1/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/r1/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)

        cognito_client = boto3.client('cognito-idp', region_name=region)   
        auth_response = cognito_client.initiate_auth(
                    ClientId=ssm_client.get_parameter(Name='/mcp_server/r1/runtime/client_id')["Parameter"]["Value"],
                    AuthFlow='USER_PASSWORD_AUTH',
                    AuthParameters={
                        'USERNAME': 'testuser',
                        'PASSWORD': 'MyPassword123!'
                    }
                )
        bearer_token=auth_response['AuthenticationResult']['AccessToken']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                    
                try:
                    print("\n👋 Testing R1 tool...")
                    search_result = await session.call_tool(
                        name="get_rapps",
                        arguments={"question": "Tell me about the rApps that you see"}
                    )
                    print(f"   Result: {search_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")

                print("\n✅ MCP tool testing completed!")

                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## Test Tool Invocation

Let's test our MCP tools by actually invoking them:

In [ ]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

## Next Steps

Now that you have successfully deployed the **R1 MCP server** to AgentCore Runtime, you will:

1. **Deploy the O2 MCP Server**: In the next module, you'll deploy `mcp_server_o2` following a similar process
2. **Create an AgentCore Gateway**: Combine both MCP servers into a unified gateway
3. **Build AI Agents**: Use these MCP servers to create intelligent agents that can manage O-RAN infrastructure

Continue to the next notebook to deploy the O2 MCP server!

# 🎉 Congratulations!

You have successfully:

✅ **Created an MCP server** with custom tools  
✅ **Tested locally** using MCP client  
✅ **Set up authentication** with Amazon Cognito  
✅ **Deployed to AWS** using AgentCore Runtime  
✅ **Invoked remotely** with proper authentication  
✅ **Learned MCP concepts** and best practices  

Your MCP server is now running on Amazon Bedrock AgentCore Runtime and ready for production use!

## Summary

In this tutorial, you learned how to:
- Build MCP servers using FastMCP
- Configure stateless HTTP transport for AgentCore compatibility
- Set up JWT authentication with Amazon Cognito
- Deploy and manage MCP servers on AWS
- Test both locally and remotely
- Use MCP clients for tool invocation

The deployed MCP server can now be integrated into larger AI applications and workflows!